In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter

In [2]:
train_CSV = "/kaggle/input/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Training/training_labels.csv"
df = pd.read_csv(train_CSV)

In [3]:
train_image = "/kaggle/input/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Training/training_words"

In [4]:
import os
import pandas as pd
import cv2
from PIL import Image
import numpy as np
import pytesseract
import tensorflow as tf
from transformers import BertTokenizerFast, TFBertForMaskedLM, BertConfig
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


In [5]:
# Define base directory where images and CSV files are stored
base_dir = "/kaggle/input/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset"  

# Image folders
train_img_dir = os.path.join(base_dir, "Training/training_words") 
val_img_dir = os.path.join(base_dir, "Validation/validation_words")
test_img_dir = os.path.join(base_dir, "Testing/testing_words")

In [6]:
train_csv = "/kaggle/input/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Training/training_labels.csv"
val_csv = "/kaggle/input/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Validation/validation_labels.csv"
test_csv = "/kaggle/input/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Testing/testing_labels.csv"

# **Tesseract**

In [7]:
def preprocess_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (128, 32))
    _, img = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return img

In [8]:
def extract_text_with_ocr(image_path):
    preprocessed_image = preprocess_image(image_path)
    img = Image.fromarray(preprocessed_image)
    extracted_text = pytesseract.image_to_string(img)
    return extracted_text

In [9]:
import pytesseract
def XYocr(csv,img_dir):
    df=pd.read_csv(f"{csv}")
    Y=df['MEDICINE_NAME']

    image_folder=f"{img_dir}"
    image_paths=[os.path.join(image_folder,filename) for filename in df['IMAGE']]
    X = [extract_text_with_ocr(image_path) for image_path in image_paths]

    return X,Y

In [10]:
xTrain,yTrain=XYocr(train_csv,train_img_dir)
xVal,yVal=XYocr(val_csv,val_img_dir)
xTest,yTest = XYocr(test_csv,test_img_dir)

In [ ]:
vocab_list = list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,!?")
vocab = {char: i for i, char in enumerate(vocab_list, start=1)}
vocab["[PAD]"] = 0
vocab["[MASK]"] = len(vocab) + 1
vocab["[CLS]"] = len(vocab) + 2
vocab["[SEP]"] = len(vocab) + 3
vocab["[UNK]"] = len(vocab) + 4


In [12]:
vocab_file = "vocab.txt"
with open(vocab_file, "w") as f:
    for token in vocab:
        f.write(token + "\n")

In [13]:
tokenizer = BertTokenizerFast(vocab_file=vocab_file)
tokenizer.pad_token = "[PAD]"
tokenizer.mask_token = "[MASK]"
tokenizer.cls_token = "[CLS]"
tokenizer.sep_token = "[SEP]"
tokenizer.unk_token = "[UNK]"


In [14]:
max_length = 128

# **Encoding**

In [15]:
def encode_dataset(texts, labels):
    input_encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
    label_encodings = tokenizer(labels.tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')
    return input_encodings, label_encodings

In [16]:
train_inputs, train_labels = encode_dataset(xTrain, yTrain)
val_inputs, val_labels = encode_dataset(xVal, yVal)
test_inputs, test_labels = encode_dataset(xTest, yTest)

In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": train_inputs["input_ids"],
        "attention_mask": train_inputs["attention_mask"]
    },
    train_labels["input_ids"]
)).batch(4)

In [18]:
val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": val_inputs["input_ids"],
        "attention_mask": val_inputs["attention_mask"]
    },
    val_labels["input_ids"]
)).batch(4)

In [19]:

test_dataset = tf.data.Dataset.from_tensor_slices((
    {
        "input_ids": test_inputs["input_ids"],
        "attention_mask": test_inputs["attention_mask"]
    },
    test_labels["input_ids"]
)).batch(4)

In [20]:
import tensorflow as tf
from transformers import BertConfig, TFBertForMaskedLM

# 2. Define custom metric
def masked_accuracy(y_true, y_pred):
    y_true = tf.cast(y_true, tf.int64)
    mask = tf.math.not_equal(y_true, 0)
    y_pred_ids = tf.argmax(y_pred, axis=-1, output_type=tf.int64)
    matches = tf.equal(y_true, y_pred_ids)
    matches = tf.logical_and(mask, matches)
    accuracy = tf.reduce_sum(tf.cast(matches, tf.float32)) / tf.reduce_sum(tf.cast(mask, tf.float32))
    return accuracy

In [21]:
config = BertConfig(
    vocab_size=len(vocab),
    hidden_size=256,
    num_hidden_layers=6,
    num_attention_heads=8,
    intermediate_size=512,
    max_position_embeddings=512
)

In [22]:
model = TFBertForMaskedLM(config)

In [23]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[masked_accuracy])  # 👈 Add custom metric here

# **Model Training**

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
)


Epoch 1/3
780/780 [==============================] - 193s 223ms/step - loss: 0.1045 - masked_accuracy: 0.9924 - val_loss: 0.0096 - val_masked_accuracy: 0.9991
Epoch 2/3
780/780 [==============================] - 168s 216ms/step - loss: 0.0084 - masked_accuracy: 0.9987 - val_loss: 0.0070 - val_masked_accuracy: 0.9991
Epoch 3/3
780/780 [==============================] - 169s 217ms/step - loss: 0.0066 - masked_accuracy: 0.9986 - val_loss: 0.0061 - val_masked_accuracy: 0.9991


In [26]:
model.evaluate(test_dataset)

195/195 [==============================] - 13s 64ms/step - loss: 0.0061 - masked_accuracy: 0.9991


[0.00607823021709919, 0.999098539352417]

# **Evaluation Matrix**

In [ ]:
def decode_predictions(pred_ids):
    return tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

true_labels = []
pred_labels = []

for batch in test_dataset:
    inputs, labels = batch
    predictions = model(inputs, training=False).logits
    pred_ids = tf.argmax(predictions, axis=-1)

    for true, pred in zip(labels.numpy(), pred_ids.numpy()):
        # Remove padding tokens (token_id == 0)
        true = true[true != 0]
        pred = pred[:len(true)]

        true_labels.extend(true)
        pred_labels.extend(pred)

precision = precision_score(true_labels, pred_labels, average='macro', zero_division=0)
recall = recall_score(true_labels, pred_labels, average='macro', zero_division=0)
f1 = f1_score(true_labels, pred_labels, average='macro', zero_division=0)
accuracy = accuracy_score(true_labels, pred_labels)

# Print the results
print("\nEvaluation Metrics on Test Set:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1 Score:  {f1:.4f}")


Evaluation Metrics on Test Set:
Accuracy:  0.9991
Precision: 0.9250
Recall:    0.9230
F1 Score:  0.9239


In [28]:
model.save_pretrained("saved_prescription_model")
tokenizer.save_pretrained("saved_prescription_model")

('saved_prescription_model/tokenizer_config.json',
 'saved_prescription_model/special_tokens_map.json',
 'saved_prescription_model/vocab.txt',
 'saved_prescription_model/added_tokens.json',
 'saved_prescription_model/tokenizer.json')

In [29]:
def predict_on_new_image(image_path, model):
    # OCR preprocessing
    extracted_text = extract_text_with_ocr(image_path)
    print(f"\nExtracted Text by OCR: {extracted_text.strip()}")

    inputs = tokenizer(extracted_text, return_tensors="tf", padding='max_length', truncation=True, max_length=max_length)

    # Predict
    outputs = model(inputs)
    pred_ids = tf.argmax(outputs.logits, axis=-1)

    # Decode prediction
    predicted_text = tokenizer.decode(pred_ids[0], skip_special_tokens=True)
    print(f"Predicted Text by Model: {predicted_text.strip()}")


In [ ]:
test_image_path1 = "/kaggle/input/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/123.png"
predict_on_new_image(test_image_path1, model)


Extracted Text by OCR: Bnclon
Predicted Text by Model:: Baclon


In [ ]:
test_image_path2 = "/kaggle/input/doctors-handwritten-prescription-bd-dataset/Doctor’s Handwritten Prescription BD dataset/Testing/testing_words/1.png"
predict_on_new_image(test_image_path2, model)

Extracted Text by OCR: Aceta
Predicted Text by Model:: Aceta
